# Sample QOCT calculations

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import numpy as np
import scipy as sp
import matplotlib
if not isnotebook:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
import nlopt
import qutip as qt
from qutip_qip.operations import toffoli

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers
import qocttools.target as target

In [ ]:
qocttools.about()

In [ ]:
data = []

# Introduction

In this notebook we show some simple examples of optimization done with QOCT, using the math_extra.maximize routine.

# Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.15, 0.0, 0.0)$ T

* $\vec{H}_m = (0, 0.001, 0.0)$ T

In [ ]:
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim

In [ ]:
H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
H_m = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)
H0 = GdW30.hGdW30(D, E, H)
V = GdW30.vGdW30(H_m)

In [ ]:
eigenvalues, eigenstates = H0.eigenstates()
H0 = H0.transform(eigenstates) - eigenvalues[0]

In [ ]:
# In principle, we could just transform V with the recently obtained eigenstates. Unfortunately, that
# would make the test results different in different computers, as the eigenstates can have different phases.
# Therefore, we use the V that was computed once, and stored in file "V". The commented code was used
# to generate and store V.
#V = V.transform(eigenstates)
#fileio.file_data_store("V", V, numformat = 'exp')
V = qt.Qobj(qt.fileio.file_data_read("V"))

In [ ]:
H = hamiltonians.hamiltonian(H0, [V])

In [ ]:
w = np.zeros(dim-1)
taui = np.zeros(dim-1)
for i in range(dim-1):
    w[i] = eigenvalues[i+1] - eigenvalues[i]
    taui[i] = 2.0*np.pi/w[i]
    print("Transition {:d}: w = {:f} MHz, tau = {:f} ns".format(i, w[i], 1000.0*taui[i]/(2.0*np.pi)))

# Time array definition

In [ ]:
T = 5*taui[0]
print("T = {:f} us*2*pi = {:f} ns".format(T, 1000*T/(2.0*np.pi)))
time = math_extra.timegrid(H0, T, 2.0)
print('#Time steps =', time.shape[0])

# Control function

The control function is parametrized with the Fourier expansion as follow:
\begin{equation}
    f(u, t) = \frac{1}{\sqrt{T}}u_0 + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k}\cos(\omega_kt) + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k + 1}\sin(\omega_kt),
\end{equation}
where $u_0\dots u_{2M + 1}$ are the control parameters. This way, we can compute the derivate respect any control parameter as
\begin{equation}
    \frac{\partial f}{\partial u_m}(u, t) = f(e_m, t),
\end{equation}
where $e_m$ is the set of parameters where all of them are zero except the m-th ane, that is equal to one.

This pulse parametrization is included in the typical_pulses.py file as pulse class.

In [ ]:
M = 10

omega = np.zeros(M+1)
omega[0] = 0.0
for k in range(1, M+1):
    omega[k] = (2.0*np.pi/T) * k
    print("omega[{:d}] = {:f} MHz".format(k, omega[k]))

# u = 1.0*np.random.rand((2*M + 1))
u = np.zeros(2*M+1)
u[2] = 1.0
u[3] = 1.0
f = pulses.pulse("fourier", T, u = u)

print("Amplitudes:")
print("{:f} mT".format(u[0]/np.sqrt(T)))
for m in range(1, 2*M+1):
    print("{:f} mT".format(2.0*u[m]/(np.sqrt(T))))

In [ ]:
fig, ax = plt.subplots()

ax.plot(time * 1000/(2.0*np.pi), f.fu(time, u))
ax.set_xlabel("Time (ns)")
ax.set_xlim(left = 0.0, right = time[-1]*1000/(2.0*np.pi))
ax.set_ylabel("f(t) (mT)")
if isnotebook:
    plt.show()
else:
    fig.savefig("pulse.pdf")

In [ ]:
fig, ax = plt.subplots()

fw, ws = f.fw(time, u)
#fw = f.fw(time, u, w = omega)
#ws = omega
fwabs = np.abs(fw)
ax.plot(ws[:M+1]/omega[1], fwabs[:M+1], marker = 'o')

if isnotebook:
    plt.show()
else:
    fig.savefig("pulse.pdf")

# State optimization 

The target will be the population of the first excited state.
\begin{equation}
T(\psi(T)) = \vert \psi(T)\vert 1\rangle\vert^2
\end{equation}
We will use a penalty function:
\begin{equation}
P(u) = -\lambda \int_0^T\!\!{\rm d}t\; f^2(u, t)\,,
\end{equation}
where the $\lambda$ penalty factor is set to:
\begin{equation}
\lambda = \frac{0.1}{\int_0^T\!\!{\rm d}t\; f^2(u^0, t)}\,.
\end{equation}
where $u^0$ are the parameteres of the initial guess pulse.

Therefore, the target functionl is:
\begin{equation}
F(\psi(T), u) = T(\psi(T)) + P(u)\,.
\end{equation}

## QOCT target function definition

In [ ]:
target_level = 1

lambda_ = 0.1 / sp.integrate.simps(f.fu(time)*f.fu(time), time)

def Pfunction(u):
    return - lambda_ * sp.integrate.simps(f.fu(time, u) * f.fu(time, u), time)

def dFdu(u, m):
    return - 2.0 * lambda_ * sp.integrate.simps(f.dfu(time, m, u) * f.fu(time, u), time)

def Fyu(y, u):
    return qt.expect(qt.fock_dm(dim, target_level), y) + Pfunction(u)

def dFdy(y, u):
    return qt.fock_dm(dim, target_level) * y

state_0 = qt.basis(dim, 0)

In [ ]:
#tg = target.Target('expectationvalue', dFdu = dFdu, operator = fock_dm(dim, target_level), Pu = Pfunction)
tg = target.Target('generic', Fyu = Fyu, dFdy = dFdy, dFdu = dFdu)

opt = qoct.Qoct(H, T, time.shape[0], tg, f, state_0,
                interaction_picture = True)

In [ ]:
derqoct, dernum, error, elapsed_time = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))
data.append(derqoct)

In [ ]:
x, optval, res = opt.maximize(maxeval = 20,verbose = True)
data.append(optval)

In [ ]:
f.set_parameters(x)
result = solvers.solve('sesolve', hamiltonians.hamiltonian(H0, V), f, state_0, time,
               returnQoutput = True, interaction_picture = True)
Gval = tg.Fyu(result[-1], x)

In [ ]:
print('Gval =', Gval)
print('Tval =', Gval - Pfunction(x))

In [ ]:
fig, ax = plt.subplots(1, 2)

occ = np.zeros_like(time)
for j in range(time.size):
    #occ[j] = Tfunction(result.states[j])
    occ[j] = tg.Fyu(result[j], x) - Pfunction(x)
ax[0].plot(time * 1000/(2.0*np.pi), f.fu(time, u), label = 'initial guess'),
ax[0].plot(time * 1000/(2.0*np.pi), f.fu(time, x), label = 'optimal pulse')
ax[0].set_xlabel("Time (ns)")
ax[0].set_xlim(left = 0.0, right = time[-1]*1000/(2.0*np.pi))
ax[0].set_ylabel("f(t) (mT)")
ax[0].legend()

ax[1].plot(time * 1000/(2.0*np.pi), occ)
ax[1].set_xlabel("Time (ns)")
ax[1].set_ylabel("P$_1$(t)")
ax[1].set_ylim(bottom = 0, top = 1)
ax[1].set_xlim(left = 0.0, right = time[-1]*1000/(2.0*np.pi))
if isnotebook:
    plt.show()
else:
    fig.savefig("pulse.pdf")

# Gate optimization

Now, the target will be the creation of the Toffoli quantum gate. We will not use a penalty function, but set bounds for the coefficients values.

## QOCT target function definition

In [ ]:
U_target = qt.Qobj(toffoli().full())
U_0 = qt.qeye(dim) #initial state

We will ask each frequency component not to be larger than 30 mT.

In [ ]:
maxhval = 30.0
kappa = maxhval*np.sqrt(T)/2.0
print(u, kappa)

In [ ]:
tg = target.Target('evolutionoperator', Utarget = U_target)

opt = qoct.Qoct(H, T, time.shape[0], tg, f, U_0,
                interaction_picture = True)

In [ ]:
derqoct, dernum, error, elapsed_time = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
x, optval, res = opt.maximize(maxeval = 50,
                              verbose = True,
                              algorithm = nlopt.LD_SLSQP,
                              upper_bounds = kappa * np.ones_like(u),
                              lower_bounds = -kappa * np.ones_like(u))
data.append(optval)

In [ ]:
with open("data", "w") as f:
    for i in data:
        f.write("{:.14e}\n".format(i))